In [1]:
import re
from datetime import datetime

In [2]:
import os
import geopandas as gpd
import geemap
import xml.etree.ElementTree as ET
import pandas as pd
from shapely.geometry import LineString

# constante requerida
namespace = {"kml": "http://www.opengis.net/kml/2.2"}

In [3]:
import geopandas as gpd
from pyproj import CRS
from pykml import parser
from shapely.geometry import Point, LineString

In [4]:
# Directorio donde están los archivos KML
path_kml = r'G:\vuelos\S07_NUEVA ESPERANZA DAVID SUAREZ'
contenido = os.listdir(path_kml)
len(contenido)

19

In [5]:
path_kml

'G:\\vuelos\\S07_NUEVA ESPERANZA DAVID SUAREZ'

In [6]:
import geopandas as gpd
from pyproj import CRS
from pykml import parser
from shapely.geometry import LineString
import os

In [7]:
def convertir_kmls_a_shapefile(kml_folder, shapefile_path):
    """
    Convierte varios archivos KML en una carpeta a un solo shapefile con la proyección UTM zona 20S.

    Parámetros:
    kml_folder (str): Carpeta que contiene los archivos KML.
    shapefile_path (str): Ruta donde se guardará el archivo shapefile combinado.
    """
    line_geometries = []
    fechas = []
    horas = []
    ids = []

    # Procesar cada archivo KML en la carpeta
    for filename in os.listdir(kml_folder):
        if filename.endswith('.kml'):
            kml_path = os.path.join(kml_folder, filename)
            
            # Leer el archivo KML
            with open(kml_path, 'r') as file:
                root = parser.parse(file).getroot()
            
            # Expresión regular para extraer la fecha, hora e ID
            match = re.search(r'T\d+_(\d{4})(\d{2})(\d{2})(\d{2})(\d{2})(\d{2})_(R\d+)\.kml', filename)
            if match:
                # Extraer los valores y formatearlos
                date = f"{match.group(1)}-{match.group(2)}-{match.group(3)}"
                time = f"{match.group(4)}:{match.group(5)}:{match.group(6)}"
                record_id = match.group(7)
            
            # Extraer coordenadas de Placemark y crear geometría LineString
            for placemark in root.Document.Placemark:
                coordinates = placemark.LineString.coordinates.text.strip().split()
                points = [tuple(map(float, coord.split(','))) for coord in coordinates]
                line_geometries.append(LineString(points))
                fechas.append(date)
                horas.append(time)
                ids.append(record_id)

    # Crear un GeoDataFrame
    gdf = gpd.GeoDataFrame({'fecha': fechas, 'hora': horas, 'id': ids, 'geometry': line_geometries}, crs='EPSG:4326')
    
    # Definir el sistema de referencia de coordenadas de destino (UTM zona 20S)
    crs_utm_20s = CRS("EPSG:32720")

    # Transformar las coordenadas al sistema UTM zona 20S
    gdf_utm = gdf.to_crs(crs_utm_20s)

    # Guardar el GeoDataFrame como un shapefile
    gdf_utm.to_file(shapefile_path, driver='ESRI Shapefile')

    print(f"Archivo shapefile combinado guardado en: {shapefile_path}")

In [8]:
# Ejemplo de uso
kml_folder = r'G:\vuelos\S07_NUEVA ESPERANZA DAVID SUAREZ'
shapefile_path = r'G:\vuelos\S07_NUEVA ESPERANZA DAVID SUAREZ\res.shp'
convertir_kmls_a_shapefile(kml_folder, shapefile_path)

Archivo shapefile combinado guardado en: G:\vuelos\S07_NUEVA ESPERANZA DAVID SUAREZ\res.shp
